# <center>**Game Connect Four**

**Connect Four is a two-player strategy game.
It is played using 42 tokens (usually 21 red tokens for one player and 21 black tokens for the other player), and a vertical grid that is 7 columns wide, each column can hold a maximum of 6 tokens.
The two players take turns at making moves and a move consists of a player dropping one of his/her tokens into the column of his/her choice.
When a token is dropped into a column, it falls until it hits the bottom or the top token in that column.
A player wins by creating an arrangement in which at least four of his/her tokens are aligned in a row column, or diagonal.**

**In this assignment we are going to implement the Monte Carlo Tree Search (MCTS) algorithms and a deciosion tree for each given dataset learned using ID3 procedures.**

# First Step - Preparation

Firstly we are going to install our dependencies!

In [1]:
!pip install numpy
!pip install pygame
!pip install pandas


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Rui Almeida\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Rui Almeida\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\Rui Almeida\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Now that our dependencies are installed, we can proceed to import our libraries.

In [2]:
import numpy as np
import pygame
import pandas as pd
import seaborn as sns
import sys
import time
import math
import copy
import random
import csv
from button import Button
from collections import Counter

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Second Step - Create the Board

In [3]:
WIN_X = 512
WIN_O = -512
DRAW = 0
MOVE_BONUS_X = 16
MOVE_BONUS_O = -16
SEGMENT_VALUES_O = {0: 0, 1: -1, 2: -10, 3: -50}
SEGMENT_VALUES_X = {0: 0, 1: 1, 2: 10, 3: 50}

def create_board():
    board = [[0 for _ in range(7)] for _ in range(6)]
    return board

Now we are going to create the Menu function and define the pygame definitions.

In [4]:
pygame.init()

SCREEN = pygame.display.set_mode((1280, 720))
pygame.display.set_caption("Menu")

BG = pygame.image.load("assets/Background.png")

def get_font(size):
    return pygame.font.Font("assets/font.ttf", size)

def a_star_m():
    while True:
        a_star_game()
        pygame.display.update()

def monte_carlo_m():
    while True:
        mc_game()
        pygame.display.update()
        
def minimax_m():
    while True:
        minimax_game()
        pygame.display.update()

def pvp_m():
    while True:
        pvp_game()
        pygame.display.update()

def pr_menu():
    pr_game()




def main_menu():
    while True:
        SCREEN.blit(BG, (0, 0))

        MENU_MOUSE_POS = pygame.mouse.get_pos()

        MENU_TEXT = get_font(100).render("MAIN MENU", True, "#b68f40")
        MENU_RECT = MENU_TEXT.get_rect(center=(640, 100))

        ASTAR_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(370, 250), 
                            text_input="A*", font=get_font(75), base_color="#b68f40", hovering_color="White")
        MC_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(370, 380), 
                            text_input="MONTE CARLO", font=get_font(30), base_color="#b68f40", hovering_color="White")
        PVP_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(370, 510), 
                            text_input="PVP", font=get_font(75), base_color="#b68f40", hovering_color="White")
        MINIMAX_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(910, 250),
                            text_input="MINIMAX", font=get_font(45), base_color="#b68f40", hovering_color="White")
        IA_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(910, 380),
                            text_input="IA", font=get_font(75), base_color="#b68f40", hovering_color="White")
        PR_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(910, 510),
                   text_input="PR", font=get_font(45), base_color="#b68f40", hovering_color="White")
        QUIT_BUTTON = Button(image=pygame.image.load("assets/Quit Rect.png"), pos=(640, 650),
                     text_input="SAIR", font=get_font(45), base_color="#b68f40", hovering_color="White")

        
        SCREEN.blit(MENU_TEXT, MENU_RECT)

        for button in [ASTAR_BUTTON, MC_BUTTON, PVP_BUTTON, MINIMAX_BUTTON, IA_BUTTON, PR_BUTTON, QUIT_BUTTON]:
            button.changeColor(MENU_MOUSE_POS)
            button.update(SCREEN)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if ASTAR_BUTTON.checkForInput(MENU_MOUSE_POS):
                     a_star_difficulty_menu()
                if MC_BUTTON.checkForInput(MENU_MOUSE_POS):
                    monte_carlo_difficulty_menu()
                if PVP_BUTTON.checkForInput(MENU_MOUSE_POS):
                    pvp_m()
                if MINIMAX_BUTTON.checkForInput(MENU_MOUSE_POS):
                    minimax_difficulty_menu()
                if PR_BUTTON.checkForInput(MENU_MOUSE_POS):
                    pr_menu()
                if IA_BUTTON.checkForInput(MENU_MOUSE_POS):
                    ia_vs_ia_menu()
                if QUIT_BUTTON.checkForInput(MENU_MOUSE_POS):
                    pygame.quit()
                    sys.exit()

                

        pygame.display.update()

Let's check if the selected column has space available!

In [5]:
def is_empty(board, col):
    return board[5][col] == 0

Now we are going to create a function to check for the next free line to put the next piece.

In [6]:
def check_next_empty_row(board, col):
    for r in range(6):
        if board[r][col] == 0:
            return r

Now we need a function to put that piece on the respective spot.

In [7]:
def put_piece(board,row,col,piece):
    board[row][col] = piece

Let´s create a function that returns the columns available!

In [8]:
def get_valid_locations(board):
    res=[]
    for i in range(7):
        if is_empty(board,i):
            res.append(i)
    return res

Now we need to check if the movement made is a winning move.

In [9]:
def win(piece, board):

    #check horizontal

    for c in range (4):
        for r in range(6):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True
            
    #check vertical
            
    for c in range(7):
        for r in range(3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True
            
    #check diagonal negativa
            
    for c in range(4):
        for r in range(3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True

    #check diagonal negativa
        
    for c in range(4):
        for r in range(3,6):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True

We also need to have a function in case of draw.

In [10]:
def draw(board):
    if win(1,board) or win(2,board):
        return False
    for i in range (6):
        for j in range(7):
            if board[i][j]==0:
                return False
    return True


Extra definitions for the board and window configuration.

In [11]:
# Definições de cores
BLACK = (0, 0, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
WHITE = (255, 255, 255)

# Configurações do tabuleiro
ROW_COUNT = 6
COLUMN_COUNT = 7

# Inicialização do Pygame
pygame.init()

# Configurações da janela
WIDTH=1280
HEIGHT=720
WINDOW_SIZE = (1280, 720)
screen = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption("Connect 4")
FONT = pygame.font.SysFont(None, 36)
WIN = pygame.display.set_mode((WIDTH, HEIGHT))

Now it´s time to draw the board on the Pygame.

In [12]:
def draw_board(board):
    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(SCREEN, BLUE, (c * 180.8, r * 120, 182.8, 120))
            pygame.draw.circle(SCREEN, BLACK, (c * 182.8 + 91.4, r * 105 + 60), 50)

    for c in range(COLUMN_COUNT):
        for r in range(ROW_COUNT):
            if board[r][c] == 1:
                pygame.draw.circle(SCREEN, RED, (c * 182.8 + 91.4, 586 - r * 105), 52)
            elif board[r][c] == 2:
                pygame.draw.circle(SCREEN, YELLOW, (c * 182.8 + 91.4, 586 - r * 105), 52)
    
    BACK_MOUSE_POS = pygame.mouse.get_pos()

    BACK = Button(image=None, pos=(1100, 675),
                            text_input="BACK", font = get_font(40), base_color="Black", hovering_color="Green")
    
    BACK.changeColor(BACK_MOUSE_POS)
    BACK.update(SCREEN)


    pygame.display.update()

# Function Connect Four - Player vs Player

In [13]:
def pvp_game():
    board = [[0 for _ in range(7)] for _ in range(6)] #Criar o board
    game_over = False
    turn = 0
    
    draw_board(board) #Desenhar o board no pygame

    PVP_MOUSE_POS = pygame.mouse.get_pos()


    PVP_BACK = Button(image=None, pos=(1100, 675),
                            text_input="BACK", font = get_font(40), base_color="Black", hovering_color="Green")
        
    PVP_BACK.changeColor(PVP_MOUSE_POS)
    PVP_BACK.update(SCREEN)
    
    while not game_over:
        
        PVP_MOUSE_POS = pygame.mouse.get_pos() #Obtem a posição do rato

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                if PVP_BACK.checkForInput(PVP_MOUSE_POS):
                    main_menu()

            if event.type == pygame.MOUSEBUTTONDOWN:
                posx = event.pos[0]
                col = int(posx // 182.8)

                if turn == 0: #Move do 1ºJogador
                    valid_move = False
                    while not valid_move:
                        if is_empty(board, col): #Confirma se o move pretendido é válido
                            valid_move=True
                            row = check_next_empty_row(board, col) #Vê em que fila vai cair a peça
                            put_piece(board, row, col, 1) # Coloca a peça no board

                            if win(1, board):
                                print('Player 1 wins!')
                                game_over = True 
                                draw_board(board)
                                time.sleep(5) 
                                main_menu()
                        else:
                            print("Try again!")
                else: # Move do 2ºJogador
                    valid_move = False
                    while not valid_move:
                        if is_empty(board, col):  # Confirma se o move pretendido é válido
                            valid_move=True
                            row = check_next_empty_row(board, col) # Vê em que fila vai cair a peça
                            put_piece(board, row, col, 2) # Coloca a peça no board

                            if win(2, board):
                                print('Player 2 wins!')
                                game_over = True
                                draw_board(board)
                                time.sleep(5)
                                main_menu()
                        else:
                            print("Try again!")

                turn += 1 
                turn = turn % 2 # Atualiza para ver quem joga

                draw_board(board)
    pygame.quit()
    quit()

Secondary functions raters of the board.

In [14]:
def evaluate_board(board): #Função que avalia o board
    score = 0

    for row in board: #Começa por avaliar os segmentos horizontais
        for i in range(len(row) - 3):
            segment = row[i:i+4]
            score += evaluate_segment(segment)

    for col in range(len(board[0])): #De seguida avalia os segmentos verticais
        for i in range(len(board) - 3):
            segment = [board[j][col] for j in range(i, i+4)]
            score += evaluate_segment(segment)

    for i in range(len(board) - 3): #Depois avalia os segmentos na diagonal negativa
        for j in range(len(board[0]) - 3):
            segment = [board[i+k][j+k] for k in range(4)]
            score += evaluate_segment(segment)

    for i in range(3, len(board)): #Depois avalia os segmentos na diagonal negativa
        for j in range(len(board[0]) - 3):
            segment = [board[i-k][j+k] for k in range(4)]
            score += evaluate_segment(segment)

    #Atribui os bonús segundo quem tem a vez de jogar        
    num_x = sum(row.count(1) for row in board)
    num_o = sum(row.count(2) for row in board)
    score += MOVE_BONUS_X if num_x == num_o else MOVE_BONUS_O

    return score

def evaluate_segment(segment):
    #Avalia segmentos de 4 elementos cada
    x_count = segment.count(1)
    o_count = segment.count(2)
    if x_count == 4:
        return WIN_X
    elif o_count == 4:
        return WIN_O
    elif x_count == 0:
        return SEGMENT_VALUES_O[o_count]  #Retorna a pontuação segundo a contagem de O's
    elif o_count == 0:
        return SEGMENT_VALUES_X[x_count]   #Retorna a pontuação segundo a contagem de X's
    else:
        return 0 

# Function of the Algorithm IA VS IA

In [15]:
def ia_vs_ia_menu():
    ia_options = ["Monte Carlo", "Minimax", "A*"]
    ia1_index = 0
    ia2_index = 1

    while True:
        SCREEN.blit(BG, (0, 0))
        MOUSE_POS = pygame.mouse.get_pos()

        TITLE = get_font(70).render("IA VS IA", True, "#b68f40")
        TITLE_RECT = TITLE.get_rect(center=(640, 100))
        SCREEN.blit(TITLE, TITLE_RECT)

        IA1_TEXT = get_font(40).render("IA 1: " + ia_options[ia1_index], True, "White")
        IA1_RECT = IA1_TEXT.get_rect(center=(400, 250))
        SCREEN.blit(IA1_TEXT, IA1_RECT)

        IA2_TEXT = get_font(40).render("IA 2: " + ia_options[ia2_index], True, "White")
        IA2_RECT = IA2_TEXT.get_rect(center=(880, 250))
        SCREEN.blit(IA2_TEXT, IA2_RECT)

        LEFT1 = Button(image=None, pos=(300, 250), text_input="<", font=get_font(40), base_color="White", hovering_color="Green")
        RIGHT1 = Button(image=None, pos=(500, 250), text_input=">", font=get_font(40), base_color="White", hovering_color="Green")
        LEFT2 = Button(image=None, pos=(780, 250), text_input="<", font=get_font(40), base_color="White", hovering_color="Green")
        RIGHT2 = Button(image=None, pos=(980, 250), text_input=">", font=get_font(40), base_color="White", hovering_color="Green")

        PLAY_BUTTON = Button(image=None, pos=(640, 400), text_input="JOGAR", font=get_font(50), base_color="White", hovering_color="Green")
        BACK_BUTTON = Button(image=None, pos=(640, 500), text_input="VOLTAR", font=get_font(40), base_color="White", hovering_color="Green")

        for button in [LEFT1, RIGHT1, LEFT2, RIGHT2, PLAY_BUTTON, BACK_BUTTON]:
            button.changeColor(MOUSE_POS)
            button.update(SCREEN)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if LEFT1.checkForInput(MOUSE_POS):
                    ia1_index = (ia1_index - 1) % len(ia_options)
                if RIGHT1.checkForInput(MOUSE_POS):
                    ia1_index = (ia1_index + 1) % len(ia_options)
                if LEFT2.checkForInput(MOUSE_POS):
                    ia2_index = (ia2_index - 1) % len(ia_options)
                if RIGHT2.checkForInput(MOUSE_POS):
                    ia2_index = (ia2_index + 1) % len(ia_options)
                if PLAY_BUTTON.checkForInput(MOUSE_POS):
                    ia_vs_ia_game(ia_options[ia1_index], ia_options[ia2_index])
                if BACK_BUTTON.checkForInput(MOUSE_POS):
                    return

        pygame.display.update()
    

In [16]:
def ia_vs_ia_game(ia1, ia2):
    board = [[0 for _ in range(7)] for _ in range(6)]
    game_over = False
    turn = 0

    draw_board(board)
    IA_BACK = Button(image=None, pos=(1100, 675), text_input="BACK", font=get_font(40), base_color="Black", hovering_color="Green")

    while not game_over:
        MOUSE_POS = pygame.mouse.get_pos()
        IA_BACK.changeColor(MOUSE_POS)
        IA_BACK.update(SCREEN)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if IA_BACK.checkForInput(MOUSE_POS):
                    main_menu()

        if turn == 0:
            if ia1 == "Monte Carlo":
                col = monte_carlo(board, time_limit=1)
            elif ia1 == "Minimax":
                col = minimax(board, 6, -math.inf, math.inf, True)[0]
            elif ia1 == "A*":
                col = a_star_with_level(board, 4)

            if is_empty(board, col):
                row = check_next_empty_row(board, col)
                put_piece(board, row, col, 1)
                if win(1, board):
                    print("IA 1 venceu!")
                    draw_board(board)
                    time.sleep(5)
                    main_menu()
                turn = 1
                draw_board(board)

        elif turn == 1:
            time.sleep(1)
            if ia2 == "Monte Carlo":
                col = monte_carlo(board, time_limit=1)
            elif ia2 == "Minimax":
                col = minimax(board, 6, -math.inf, math.inf, True)[0]
            elif ia2 == "A*":
                col = a_star_with_level(board, 4)

            if is_empty(board, col):
                row = check_next_empty_row(board, col)
                put_piece(board, row, col, 2)
                if win(2, board):
                    print("IA 2 venceu!")
                    draw_board(board)
                    time.sleep(5)
                    main_menu()
                turn = 0
                draw_board(board)


# Function of the Algorithm A-Star

In [17]:
def a_star(board): #Função do algoritmo A-Star
    best_move = None
    best_score = -512

    for column in range(0,7): #Para cada coluna, se for possível jogar, avalia o board e calcula a melhor jogada
        if is_empty(board, column):
            row = check_next_empty_row(board, column)
            new_board = copy.deepcopy(board) #Faz uma cópia do tabuleiro de forma a que não afete o original
            new_board[row][column] = 2
            score = evaluate_board(new_board)
            print(score)
            if score > best_score: #Atualiza o best score segundo a melhor jogada
                best_score = score
                best_move = column

    return best_move # Retorna a jogada ideal segundo o algoritmo

def a_star_with_level(board, level):
    best_move = None
    best_score = -512

    for column in range(7):
        if is_empty(board, column):
            row = check_next_empty_row(board, column)
            new_board = copy.deepcopy(board)
            new_board[row][column] = 2
            score = evaluate_board(new_board) + (level * 10)  # valoriza um pouco mais no difícil

            if score > best_score:
                best_score = score
                best_move = column

    return best_move


def a_star_game_with_difficulty(level):
    board = [[0 for _ in range(7)] for _ in range(6)]
    game_over = False
    turn = 0

    draw_board(board)

    ASTAR_BACK = Button(image=None, pos=(1100, 675),
                        text_input="BACK", font=get_font(40), base_color="Black", hovering_color="Green")

    while not game_over:
        MOUSE_POS = pygame.mouse.get_pos()
        ASTAR_BACK.changeColor(MOUSE_POS)
        ASTAR_BACK.update(SCREEN)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                if ASTAR_BACK.checkForInput(MOUSE_POS):
                    main_menu()

            if turn == 0 and event.type == pygame.MOUSEBUTTONDOWN:
                posx = event.pos[0]
                col = int(posx // 182.8)
                if is_empty(board, col):
                    row = check_next_empty_row(board, col)
                    put_piece(board, row, col, 1)
                    if win(1, board):
                        print("Player 1 wins!")
                        draw_board(board)
                        time.sleep(5)
                        main_menu()
                    turn = 1
                    draw_board(board)

        if turn == 1:
            time.sleep(1)
            col = a_star_with_level(board, level)
            if is_empty(board, col):
                row = check_next_empty_row(board, col)
                put_piece(board, row, col, 2)
                if win(2, board):
                    print("IA wins!")
                    draw_board(board)
                    time.sleep(5)
                    main_menu()
                turn = 0
                draw_board(board)

    time.sleep(5)
    pygame.quit()
    quit()

def a_star_easy():
    a_star_game_with_difficulty(1)

def a_star_medium():
    a_star_game_with_difficulty(2)

def a_star_hard():
    a_star_game_with_difficulty(3)


In [18]:
def evaluate_window(window, piece):
    score = 0
    opponent_piece = 1 if piece == 2 else 2

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(0) == 1:
        score += 10
    elif window.count(piece) == 2 and window.count(0) == 2:
        score += 5

    if window.count(opponent_piece) == 3 and window.count(0) == 1:
        score -= 80  # prioridade em bloquear o adversário

    return score

def evaluate_board(board, piece=2):
    score = 0

    # Centro do tabuleiro
    center_col = [board[i][3] for i in range(6)]
    center_score = center_col.count(piece)
    score += center_score * 6

    # Avaliação horizontal
    for row in board:
        for c in range(4):
            window = row[c:c+4]
            score += evaluate_window(window, piece)

    # Avaliação vertical
    for c in range(7):
        col_array = [board[r][c] for r in range(6)]
        for r in range(3):
            window = col_array[r:r+4]
            score += evaluate_window(window, piece)

    # Diagonal positiva
    for r in range(3):
        for c in range(4):
            window = [board[r+i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    # Diagonal negativa
    for r in range(3):
        for c in range(4):
            window = [board[r+3-i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    return score


In [19]:
WINDOW_LENGTH=4
COLUMN_COUNT=7
def score_position(board, piece):
	score = 0

	## Score center column
	center_array=[]
	for i in range(6):
		center_array.append(board[i][3])
	center_count = center_array.count(piece)
	score += center_count * 6

	## Score Horizontal
	for r in range(ROW_COUNT):
		row_array = board[r]
		for c in range(COLUMN_COUNT-3):
			window = row_array[c:c+WINDOW_LENGTH]
			score += evaluate_window(window, piece)

	## Score Vertical
	for c in range(COLUMN_COUNT):
		col_array = []
		for i in range(6):
			col_array.append(board[i][c])
		for r in range(ROW_COUNT-3):
			window = col_array[r:r+WINDOW_LENGTH]
			score += evaluate_window(window, piece)

	## Score posiive sloped diagonal
	for r in range(ROW_COUNT-3):
		for c in range(COLUMN_COUNT-3):
			window = [board[r+i][c+i] for i in range(WINDOW_LENGTH)]
			score += evaluate_window(window, piece)

	for r in range(ROW_COUNT-3):
		for c in range(COLUMN_COUNT-3):
			window = [board[r+3-i][c+i] for i in range(WINDOW_LENGTH)]
			score += evaluate_window(window, piece)

	return score

# Function of the Algorithm Minimax

In [20]:
def minimax(board, depth, alpha, beta, maximizingPlayer):
	valid_locations = get_valid_locations(board)
	is_terminal = win(1, board) or win(2, board) or draw(board)
	if depth == 0 or is_terminal:
		if is_terminal:
			if win(2,board):
				return (None, 100000000000000)
			elif win(1,board):
				return (None, -10000000000000)
			else: # Não há mais moves válidos. Jogo acabou.
				return (None, 0)
		else: # Depht é zero
			return (None, score_position(board, 2))
	if maximizingPlayer:
		value = -math.inf
		column = random.choice(valid_locations)
		for col in valid_locations:
			row = check_next_empty_row(board, col) # Verifica a fila em que a peça irá cair
			b_copy = copy.deepcopy(board) # Copia o board de forma a não afetar o original
			put_piece(b_copy, row, col, 2) # Coloca a peça em questão
			new_score = minimax(b_copy, depth-1, alpha, beta, False)[1] # Calcula o score
			if new_score > value:
				value = new_score # Atualiza os valores dos scores
				column = col
			alpha = max(alpha, value)
			if alpha >= beta:
				break
		return column, value

	else: # Jogador Maximizante 
		value = math.inf
		column = random.choice(valid_locations)
		for col in valid_locations:
			row = check_next_empty_row(board, col) # Verifica a fila em que a peça irá cair
			b_copy = copy.deepcopy(board)
			put_piece(b_copy, row, col, 1) # Coloca a peça em questão
			new_score = minimax(b_copy, depth-1, alpha, beta, True)[1] # Calcula o score
			if new_score < value:
				value = new_score # Atualiza os valores dos scores
				column = col
			beta = min(beta, value)
			if alpha >= beta:
				break
		return column, value

# Function of the Algorithm Monte Carlo

In [ ]:
class MCTSNode:
    def __init__(self, board, move=None, parent=None, player_num=1):
        self.board = board
        self.move = move
        self.parent = parent
        self.children = {}
        self.wins = 0
        self.visits = 0
        self.untried_moves = possible_moves(board)
        self.player_num = player_num

    def UCB1(self, total_visits, C=2):
        if self.visits == 0:
            return float('inf')
        win_rate = self.wins / self.visits
        exploration_factor = C * math.sqrt((math.log(total_visits)) / self.visits)
        return win_rate + exploration_factor

    def select_child(self):
        return max(self.children.values(), key=lambda child: child.UCB1(self.visits))

    def add_child(self, move, player_num):
        new_board = np.copy(self.board)
        row = check_next_empty_row(new_board, move)
        put_piece(new_board, row, move, player_num)
        child_node = MCTSNode(new_board, move=move, parent=self, player_num=3 - player_num)
        self.untried_moves.remove(move)
        self.children[move] = child_node
        return child_node

    def update(self, result):
        self.visits += 1
        if result == self.player_num:
            self.wins += 1
        elif result != 0:
            self.wins -= 1

def possible_moves(board):
    return [col for col in range(7) if is_empty(board, col)]

def simulate(board, player_num):
    temp_board = np.copy(board)
    current_player = player_num

    while True:
        moves = possible_moves(temp_board)
        if not moves:
            return 0  # Draw

        move = random.choice(moves)
        row = check_next_empty_row(temp_board, move)
        put_piece(temp_board, row, move, current_player)

        if win(current_player, temp_board):
            return current_player  # Return winning player

        current_player = 3 - current_player

def MCTS(root, time_limit=1):
    start_time = time.time()
    while (time.time() - start_time) < time_limit:
        node = root

        # Selection
        while node.untried_moves == [] and node.children:
            node = node.select_child()

        # Expansion
        if node.untried_moves:
            move = random.choice(node.untried_moves)
            node = node.add_child(move, node.player_num)

        # Simulation
        current_result = simulate(node.board, node.player_num)

        # Backpropagation
        while node is not None:
            node.update(current_result)
            node = node.parent

def monte_carlo_difficulty(board, player_num=1, simulations=100):
    root = MCTSNode(np.copy(board), move=None, parent=None, player_num=player_num)
    simulations_done = 0

    while simulations_done < simulations:
        node = root

        # Seleção
        while node.children and not node.untried_moves:
            node = node.select_child()

        # Expansão
        if node.untried_moves:
            move = random.choice(node.untried_moves)
            node = node.add_child(move, node.player_num)

        # Simulação
        result = simulate(node.board, node.player_num)

        # Retropropagação
        while node is not None:
            node.update(result)
            node = node.parent

        simulations_done += 1

    # Escolher a jogada mais visitada
    valid_children = [child for child in root.children.values() if child.visits > 0]
    if not valid_children:
        return random.choice(possible_moves(board))

    best_move = max(valid_children, key=lambda x: (x.visits, -abs(x.move - 3))).move
    return best_move


In [22]:
def monte_carlo(board, player_num=1, time_limit=1):
    root = MCTSNode(np.copy(board), move=None, parent=None, player_num=player_num)
    MCTS(root, time_limit)
    valid_children = [child for child in root.children.values() if child.visits > 0]
    if not valid_children:
        return random.choice(possible_moves(board))
    
    best_move = max(valid_children, key=lambda x: (x.visits, -abs(x.move - 3))).move
    return best_move


In [ ]:
def monte_carlo_game_custom(simulations):
    board = [[0 for _ in range(7)] for _ in range(6)]
    game_over = False
    turn = 0
    draw_board(board)

    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

            if turn == 0 and event.type == pygame.MOUSEBUTTONDOWN:
                posx = event.pos[0]
                col = int(posx // 182.8)

                if is_empty(board, col):
                    row = check_next_empty_row(board, col)
                    put_piece(board, row, col, 1)

                    if win(1, board):
                        print("Jogador vence!")
                        game_over = True

                    turn = 1
                    draw_board(board)

        if turn == 1 and not game_over:
            time.sleep(0.5)
            col = monte_carlo_difficulty(board, player_num=2, simulations=simulations)

            if is_empty(board, col):
                row = check_next_empty_row(board, col)
                put_piece(board, row, col, 2)

                if win(2, board):
                    print("IA vence!")
                    game_over = True

                turn = 0
                draw_board(board)

    time.sleep(3)


In [ ]:
def mc_easy():
    monte_carlo_game_custom(simulations=30)

def mc_medium():
    monte_carlo_game_custom(simulations=100)

def mc_hard():
    monte_carlo_game_custom(simulations=500)


# Function Connnect Four - Player vs A-Star

In [23]:
def a_star_game():
        
    board = [[0 for _ in range(7)] for _ in range(6)]
    
    game_over = False
    turn = 0

    draw_board(board)

    ASTAR_MOUSE_POS = pygame.mouse.get_pos()


    ASTAR_BACK = Button(image=None, pos=(1100, 675),
                            text_input="BACK", font = get_font(40), base_color="Black", hovering_color="Green")
        
    ASTAR_BACK.changeColor(ASTAR_MOUSE_POS)
    ASTAR_BACK.update(SCREEN)

    while not game_over:

        ASTAR_MOUSE_POS = pygame.mouse.get_pos()
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if ASTAR_BACK.checkForInput(ASTAR_MOUSE_POS):
                    main_menu()

            if turn == 0 and event.type == pygame.MOUSEBUTTONDOWN:
                # Get the x position of the mouse
                posx = event.pos[0]
                col = int(posx // 182.8)

                if is_empty(board, col):
                    row = check_next_empty_row(board, col)
                    put_piece(board, row, col, 1)

                    if win(1, board):
                        print('Player 1 wins!')
                        game_over = True
                        draw_board(board)
                        time.sleep(5)
                        main_menu()

                    turn = 1

                    draw_board(board)

        # AI player's turn
        if turn == 1:
            time.sleep(1)
            col=a_star(board)
            if is_empty(board, col):
                row = check_next_empty_row(board, col)
                put_piece(board, row, col, 2)

                if win(2, board):
                    print('Player 2 wins!')
                    game_over = True
                    draw_board(board)
                    time.sleep(5)
                    main_menu()

                turn = 0

                draw_board(board)
       
    time.sleep(10)
    
    pygame.quit()
    quit()


# Function Connect Four - Player vs Minimax

In [24]:
def minimax_game_with_difficulty(depth):
    board = [[0 for _ in range(7)] for _ in range(6)]
    game_over = False
    turn = 0

    draw_board(board)

    BACK = Button(image=None, pos=(1100, 675),
                  text_input="BACK", font=get_font(40), base_color="Black", hovering_color="Green")

    while not game_over:
        mouse_pos = pygame.mouse.get_pos()
        BACK.changeColor(mouse_pos)
        BACK.update(SCREEN)  # usa a variável SCREEN global do projeto

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                if BACK.checkForInput(mouse_pos):
                    main_menu()

            if turn == 0 and event.type == pygame.MOUSEBUTTONDOWN:
                posx = event.pos[0]
                col = int(posx // 182.8)
                if is_empty(board, col):
                    row = check_next_empty_row(board, col)
                    put_piece(board, row, col, 1)
                    if win(1, board):
                        print("Player 1 wins!")
                        draw_board(board)
                        time.sleep(5)
                        main_menu()
                    turn = 1
                    draw_board(board)

        if turn == 1:
            time.sleep(1)
            col = minimax(board, depth, -math.inf, math.inf, True)[0]
            if is_empty(board, col):
                row = check_next_empty_row(board, col)
                put_piece(board, row, col, 2)
                if win(2, board):
                    print("IA wins!")
                    draw_board(board)
                    time.sleep(5)
                    main_menu()
                turn = 0
                draw_board(board)
    time.sleep(5)
    pygame.quit()
    quit()

def minimax_easy():
    minimax_game_with_difficulty(2)

def minimax_medium():
    minimax_game_with_difficulty(4)

def minimax_hard():
    minimax_game_with_difficulty(6)


# Function Connect Four - Player vs Monte Carlo

In [25]:
def mc_game():
        
    board = [[0 for _ in range(7)] for _ in range(6)]
    
    game_over = False
    turn = 0

    draw_board(board)

    MC_MOUSE_POS = pygame.mouse.get_pos()


    MC_BACK = Button(image=None, pos=(1100, 675),
                            text_input="BACK", font = get_font(40), base_color="Black", hovering_color="Green")
        
    MC_BACK.changeColor(MC_MOUSE_POS)
    MC_BACK.update(SCREEN)

    while not game_over:

        MC_MOUSE_POS = pygame.mouse.get_pos()

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                if MC_BACK.checkForInput(MC_MOUSE_POS):
                    main_menu()

            if turn == 0 and event.type == pygame.MOUSEBUTTONDOWN:
                # Get the x position of the mouse
                posx = event.pos[0]
                col = int(posx // 182.8)

                if is_empty(board, col):
                    row = check_next_empty_row(board, col)
                    put_piece(board, row, col, 1)

                    if win(1, board):
                        print(1)
                        game_over = True
                        draw_board(board)
                        time.sleep(5)
                        main_menu()

                    turn = 1

                    draw_board(board)

        # AI player's turn
        if turn == 1:
            col=monte_carlo(board)
            if is_empty(board, col):
                row = check_next_empty_row(board, col)
                put_piece(board, row, col, 2)

                if win(2, board):
                    print(1)
                    game_over = True
                    draw_board(board)
                    time.sleep(5)
                    main_menu()

                turn = 0

                draw_board(board)
    time.sleep(10)
    pygame.quit()



# **Now let´s create the menus of difficulty of each algorithm**

# MiniMax Difficulty Menu

In [26]:
def minimax_difficulty_menu():
    while True:
        SCREEN.blit(BG, (0, 0))
        MOUSE_POS = pygame.mouse.get_pos()

        MENU_TEXT = get_font(80).render("MINIMAX - Dificuldade", True, "#b68f40")
        MENU_RECT = MENU_TEXT.get_rect(center=(640, 100))
        SCREEN.blit(MENU_TEXT, MENU_RECT)

        EASY_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(370, 250),
                             text_input="FÁCIL", font=get_font(45), base_color="#d7fcd4", hovering_color="White")
        MEDIUM_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(370, 350),
                               text_input="MÉDIO", font=get_font(45), base_color="#d7fcd4", hovering_color="White")
        HARD_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(370, 450),
                             text_input="DIFÍCIL", font=get_font(45), base_color="#d7fcd4", hovering_color="White")
        BACK_BUTTON = Button(image=pygame.image.load("assets/Quit Rect.png"), pos=(910, 600),
                             text_input="VOLTAR", font=get_font(45), base_color="#d7fcd4", hovering_color="White")

        for button in [EASY_BUTTON, MEDIUM_BUTTON, HARD_BUTTON, BACK_BUTTON]:
            button.changeColor(MOUSE_POS)
            button.update(SCREEN)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if EASY_BUTTON.checkForInput(MOUSE_POS):
                    minimax_easy()
                if MEDIUM_BUTTON.checkForInput(MOUSE_POS):
                    minimax_medium()
                if HARD_BUTTON.checkForInput(MOUSE_POS):
                    minimax_hard()
                if BACK_BUTTON.checkForInput(MOUSE_POS):
                    return

        pygame.display.update()


# Monte Carlo Difficulty Menu

In [27]:
def monte_carlo_difficulty_menu():
    while True:
        SCREEN.blit(BG, (0, 0))
        MOUSE_POS = pygame.mouse.get_pos()

        MENU_TEXT = get_font(80).render("MONTE CARLO - Dificuldade", True, "#b68f40")
        MENU_RECT = MENU_TEXT.get_rect(center=(640, 100))
        SCREEN.blit(MENU_TEXT, MENU_RECT)

        EASY_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(370, 250),
                             text_input="FÁCIL", font=get_font(45), base_color="#d7fcd4", hovering_color="White")
        MEDIUM_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(370, 350),
                               text_input="MÉDIO", font=get_font(45), base_color="#d7fcd4", hovering_color="White")
        HARD_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(370, 450),
                             text_input="DIFÍCIL", font=get_font(45), base_color="#d7fcd4", hovering_color="White")
        BACK_BUTTON = Button(image=pygame.image.load("assets/Quit Rect.png"), pos=(910, 600),
                             text_input="VOLTAR", font=get_font(45), base_color="#d7fcd4", hovering_color="White")

        for button in [EASY_BUTTON, MEDIUM_BUTTON, HARD_BUTTON, BACK_BUTTON]:
            button.changeColor(MOUSE_POS)
            button.update(SCREEN)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if EASY_BUTTON.checkForInput(MOUSE_POS):
                    mc_easy()
                if MEDIUM_BUTTON.checkForInput(MOUSE_POS):
                    mc_medium()
                if HARD_BUTTON.checkForInput(MOUSE_POS):
                    mc_hard()
                if BACK_BUTTON.checkForInput(MOUSE_POS):
                    return

        pygame.display.update()


# A-Star Difficulty Menu

In [28]:
def a_star_difficulty_menu():
    while True:
        SCREEN.blit(BG, (0, 0))
        MOUSE_POS = pygame.mouse.get_pos()

        MENU_TEXT = get_font(80).render("A* - Dificuldade", True, "#b68f40")
        MENU_RECT = MENU_TEXT.get_rect(center=(640, 100))
        SCREEN.blit(MENU_TEXT, MENU_RECT)

        EASY_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(370, 250),
                             text_input="FÁCIL", font=get_font(45), base_color="#d7fcd4", hovering_color="White")
        MEDIUM_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(370, 350),
                               text_input="MÉDIO", font=get_font(45), base_color="#d7fcd4", hovering_color="White")
        HARD_BUTTON = Button(image=pygame.image.load("assets/Play Rect.png"), pos=(370, 450),
                             text_input="DIFÍCIL", font=get_font(45), base_color="#d7fcd4", hovering_color="White")
        BACK_BUTTON = Button(image=pygame.image.load("assets/Quit Rect.png"), pos=(910, 600),
                             text_input="VOLTAR", font=get_font(45), base_color="#d7fcd4", hovering_color="White")

        for button in [EASY_BUTTON, MEDIUM_BUTTON, HARD_BUTTON, BACK_BUTTON]:
            button.changeColor(MOUSE_POS)
            button.update(SCREEN)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN:
                if EASY_BUTTON.checkForInput(MOUSE_POS):
                    a_star_easy()
                if MEDIUM_BUTTON.checkForInput(MOUSE_POS):
                    a_star_medium()
                if HARD_BUTTON.checkForInput(MOUSE_POS):
                    a_star_hard()
                if BACK_BUTTON.checkForInput(MOUSE_POS):
                    return

        pygame.display.update()


# Decision Tree (ID3)

In [29]:
def calcular_entropia(coluna):
    total = len(coluna)
    contador = Counter(coluna)
    entropia = 0

    for classe in contador:
        prob = contador[classe] / total
        entropia -= prob * math.log2(prob)

    return entropia


def ganho_informacao(df, atributo, alvo):
    entropia_total = calcular_entropia(df[alvo])
    valores_unicos = df[atributo].unique()
    entropia_atributo = 0

    for valor in valores_unicos:
        subconjunto = df[df[atributo] == valor]
        peso = len(subconjunto) / len(df)
        entropia_atributo += peso * calcular_entropia(subconjunto[alvo])

    ganho = entropia_total - entropia_atributo
    return ganho



Let´s check our dataset and do the discretization!

In [30]:
# Carregar o dataset Iris
iris = sns.load_dataset("iris")

# Função para discretizar em 3 categorias
def discretize_column(col, labels=["low", "medium", "high"]):
    thresholds = col.quantile([0.33, 0.66]).values
    return pd.cut(col, bins=[-float("inf"), thresholds[0], thresholds[1], float("inf")], labels=labels)

# Discretizar todas as 4 colunas numéricas
iris["sepal_length_cat"] = discretize_column(iris["sepal_length"])
iris["sepal_width_cat"] = discretize_column(iris["sepal_width"])
iris["petal_length_cat"] = discretize_column(iris["petal_length"])
iris["petal_width_cat"] = discretize_column(iris["petal_width"])

# Novo dataframe com colunas discretizadas + target
iris_discrete = iris[[
    "sepal_length_cat", "sepal_width_cat",
    "petal_length_cat", "petal_width_cat",
    "species"
]].copy()

# Confirmar discretização
print(iris_discrete.head())


  sepal_length_cat sepal_width_cat petal_length_cat petal_width_cat species
0              low            high              low             low  setosa
1              low          medium              low             low  setosa
2              low          medium              low             low  setosa
3              low          medium              low             low  setosa
4              low            high              low             low  setosa


In [31]:
def id3(df, atributos, alvo):
    classes = df[alvo].unique()

    # CASO BASE 1: se todas as instâncias são da mesma classe
    if len(classes) == 1:
        return classes[0]

    # CASO BASE 2: se não restam atributos, retorna classe mais comum
    if len(atributos) == 0:
        return df[alvo].mode()[0]

    # Escolher o melhor atributo com base no ganho de informação
    ganhos = {atrib: ganho_informacao(df, atrib, alvo) for atrib in atributos}
    melhor_atributo = max(ganhos, key=ganhos.get)

    arvore = {melhor_atributo: {}}
    valores_unicos = df[melhor_atributo].unique()

    for valor in valores_unicos:
        subconjunto = df[df[melhor_atributo] == valor]
        if subconjunto.empty:
            arvore[melhor_atributo][valor] = df[alvo].mode()[0]
        else:
            novos_atributos = [a for a in atributos if a != melhor_atributo]
            arvore[melhor_atributo][valor] = id3(subconjunto, novos_atributos, alvo)

    return arvore


In [32]:
# Discretizar rapidamente para teste
iris = sns.load_dataset("iris")
iris["petal_length"] = pd.cut(iris["petal_length"], bins=3, labels=["curta", "media", "longa"])
iris["petal_width"] = pd.cut(iris["petal_width"], bins=3, labels=["estreita", "média", "larga"])
iris_attributes = ["sepal_length_cat", "sepal_width_cat", "petal_length_cat", "petal_width_cat"]
iris_target = "species"

arvore = id3(iris_discrete, iris_attributes, iris_target)

print(arvore)


{'petal_width_cat': {'low': 'setosa', 'medium': {'petal_length_cat': {'medium': 'versicolor', 'high': {'sepal_length_cat': {'high': {'sepal_width_cat': {'medium': 'versicolor', 'low': 'versicolor'}}, 'medium': {'sepal_width_cat': {'low': 'virginica'}}}}}}, 'high': {'sepal_width_cat': {'medium': {'petal_length_cat': {'medium': {'sepal_length_cat': {'medium': 'versicolor'}}, 'high': {'sepal_length_cat': {'high': 'virginica', 'medium': 'virginica'}}}}, 'high': 'virginica', 'low': 'virginica'}}}}


In [33]:
def classificar_exemplo(exemplo, arvore):
    if not isinstance(arvore, dict):
        return arvore  # chegaste a uma folha (classe final)

    atributo = next(iter(arvore))  # o primeiro (e único) atributo neste nível
    valor = exemplo[atributo]

    if valor not in arvore[atributo]:
        return "Classe desconhecida"  # caso de valor nunca visto

    ramo = arvore[atributo][valor]
    return classificar_exemplo(exemplo, ramo)


In [34]:
def testar_arvore(df_teste, arvore, alvo):
    corretos = 0
    total = len(df_teste)

    for i, linha in df_teste.iterrows():
        previsto = classificar_exemplo(linha, arvore)
        real = linha[alvo]
        if previsto == real:
            corretos += 1

    precisao = corretos / total
    return precisao


In [35]:
# Dividir o dataset em treino e teste manualmente (sem sklearn)
iris_shuffled = iris_discrete.sample(frac=1, random_state=42).reset_index(drop=True)
split_index = int(0.7 * len(iris_shuffled))

iris_train = iris_shuffled.iloc[:split_index]
iris_test = iris_shuffled.iloc[split_index:]

# Definir atributos e alvo
iris_attributes = ["sepal_length_cat", "sepal_width_cat", "petal_length_cat", "petal_width_cat"]
iris_target = "species"

# Treinar a árvore com ID3
iris_tree = id3(iris_train, iris_attributes, iris_target)

# Avaliar nos dados de teste
acc = testar_arvore(iris_test, iris_tree, iris_target)
print(f"Precisão da árvore de decisão (ID3) no Iris: {acc:.2%}")


Precisão da árvore de decisão (ID3) no Iris: 91.11%


In [36]:
def codificar_tabuleiro(board):
    # Converte a matriz 6x7 numa string contínua com 42 caracteres
    return ''.join(str(cell) for row in board for cell in row)


In [37]:
def gerar_dataset_mcts(n_jogadas=500, filename="dataset_connect4.csv"):
    dados = []

    for j in range(n_jogadas):
        board = [[0 for _ in range(7)] for _ in range(6)]
        game_over = False
        turn = 0
        turn_count = 0

        while not game_over and turn_count < 42:
            turn_count += 1

            try:
                col = monte_carlo(board, time_limit=0.1)
            except:
                continue  # se der erro, tenta outra jogada

            jogador = 1 if turn == 0 else 2

            if col not in range(7):
                continue  # jogada inválida

            if not is_empty(board, col):
                continue  # coluna cheia

            row = check_next_empty_row(board, col)
            put_piece(board, row, col, jogador)

            estado = codificar_tabuleiro(board)
            dados.append({"estado": estado, "jogada": col})

            if win(jogador, board):
                game_over = True
            else:
                turn = 1 - turn

        if j % 100 == 0:
            print(f"{j} jogos simulados...")

    with open(filename, mode='w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=["estado", "jogada"])
        writer.writeheader()
        writer.writerows(dados)

    print(f"{len(dados)} jogadas guardadas em {filename}")


In [ ]:
gerar_dataset_mcts(1200)


0 jogos simulados...
100 jogos simulados...
200 jogos simulados...


In [ ]:
# Carregar o dataset
df_c4 = pd.read_csv("dataset_connect4.csv")

# Separar a string "estado" em 42 colunas individuais
for i in range(42):
    df_c4[f"pos_{i}"] = df_c4["estado"].str[i]

# Remover a coluna original "estado"
df_c4 = df_c4.drop(columns=["estado"])

# Ver as primeiras linhas
df_c4.head()


NameError: name 'pd' is not defined

In [ ]:
c4_attributes = [f"pos_{i}" for i in range(42)]
c4_target = "jogada"

c4_tree_full = id3(df_c4, c4_attributes, c4_target)
print("Árvore de decisão do Connect4 treinada com sucesso!")


Árvore de decisão do Connect4 treinada com sucesso!


In [ ]:
def prever_jogada_com_arvore(board, arvore):
    estado = codificar_tabuleiro(board)
    exemplo = {}

    for i, valor in enumerate(estado):
        exemplo[f"pos_{i}"] = valor

    jogada_prevista = classificar_exemplo(exemplo, arvore)

    try:
        col = int(jogada_prevista)
        if 0 <= col < 7 and is_empty(board, col):
            return col
        else:
            return random.choice([i for i in range(7) if is_empty(board, i)])
    except:
        return random.choice([i for i in range(7) if is_empty(board, i)])

    
    col = int(jogada_prevista)
    if 0 <= col < 7 and is_empty(board, col):
        return col
    else:
        # Escolher aleatoriamente se for inválida
        return random.choice([i for i in range(7) if is_empty(board, i)])


In [ ]:
def pr_game():
    board = [[0 for _ in range(7)] for _ in range(6)]
    game_over = False
    turn = 0  # 0 = jogador, 1 = IA PR

    draw_board(board)

    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

            if turn == 0 and event.type == pygame.MOUSEBUTTONDOWN:
                posx = event.pos[0]
                col = int(posx // 182.8)

                if is_empty(board, col):
                    row = check_next_empty_row(board, col)
                    put_piece(board, row, col, 1)

                    if win(1, board):
                        print("Jogador vence!")
                        game_over = True

                    turn = 1
                    draw_board(board)

        if turn == 1 and not game_over:
            time.sleep(1)
            col = prever_jogada_com_arvore(board, c4_tree_full)

            if is_empty(board, col):
                row = check_next_empty_row(board, col)
                put_piece(board, row, col, 2)

                if win(2, board):
                    print("IA PR vence!")
                    game_over = True

                turn = 0
                draw_board(board)

    time.sleep(5)
    return



In [ ]:
# Embaralhar e dividir manualmente
df_c4_shuffled = df_c4.sample(frac=1, random_state=42).reset_index(drop=True)
split_index = int(0.7 * len(df_c4_shuffled))

df_c4_train = df_c4_shuffled.iloc[:split_index]
df_c4_test = df_c4_shuffled.iloc[split_index:]

# Treinar nova árvore com dados de treino
c4_tree_test = id3(df_c4_train, c4_attributes, c4_target)

# Avaliação com dados de teste
print(f"Tamanho do df_c4_test: {len(df_c4_test)}")
print(df_c4_test.head())
print(f"Tamanho do df_c4 original: {len(df_c4)}")
print(df_c4.head())


acc_c4 = testar_arvore(df_c4_test, c4_tree_test, c4_target)
print(f"Acurácia real da árvore do Connect4 (validação): {acc_c4:.2%}")


Tamanho do df_c4_test: 12051
       jogada pos_0 pos_1 pos_2 pos_3 pos_4 pos_5 pos_6 pos_7 pos_8  ...  \
28116       6     2     2     2     0     0     1     1     1     1  ...   
28117       5     1     2     1     0     0     1     1     2     2  ...   
28118       2     2     1     2     1     2     2     1     1     2  ...   
28119       1     2     1     1     0     0     2     1     1     2  ...   
28120       1     2     1     1     0     0     2     2     1     2  ...   

      pos_32 pos_33 pos_34 pos_35 pos_36 pos_37 pos_38 pos_39 pos_40 pos_41  
28116      0      0      0      0      0      0      0      0      0      0  
28117      0      1      0      0      0      0      0      0      2      0  
28118      0      2      0      0      2      0      0      0      1      0  
28119      0      0      2      0      0      0      0      0      0      1  
28120      0      0      0      0      0      0      0      0      0      0  

[5 rows x 43 columns]
Tamanho do df_c4 origin

In [ ]:
def reset_pygame():
    pygame.init()
    global SCREEN, BG
    SCREEN = pygame.display.set_mode((1280, 720))
    BG = pygame.image.load("assets/Background.png")  # ajusta se usares outro nome


In [ ]:
reset_pygame()
main_menu()


NameError: name 'pygame' is not defined